In [ ]:
# !pip install tensorflow==2.4.1
# !pip install keras==2.3.1

In [ ]:
import os
import sys
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import re
import time
import torch, gc
import datetime
import nltk

from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
#from tensorflow.python.keras.preprocessing.text import Tokenizer"
from keras.preprocessing.sequence import pad_sequences
#from tensorflow.python.keras.preprocessing.sequence import pad_sequences"
from tensorflow.python.training import training
from tensorflow.python.client import device_lib
from google.colab import drive
from google.colab import files
from tqdm import tqdm

Using TensorFlow backend.


In [ ]:
try:
  drive.mount('/content/drive')
  os.chdir("drive/My Drive/GoogleColab/")
  print('Dosya dizini belirtilen konuma ayarlandı.')
except:
  print('Dosya dizini belirtilen konumda...')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dosya dizini belirtilen konuma ayarlandı.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jul  3 16:31:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
try:
  %load_ext tensorboard
  print("TensorBoard eklentisi kuruldu.")
finally:
  %rm -rf ./logs/
  print("Önceki çalıştırmalardan elde edilen günlükler temizlendi.")

TensorBoard eklentisi kuruldu.
Önceki çalıştırmalardan elde edilen günlükler temizlendi.


In [ ]:
if tf.executing_eagerly():
  print("İstekli çalışma modu çalışıyor.")
else:
  tf.compat.v1.enable_eager_execution()
  print("İstekli çalışma moduna geçildi.")

İstekli çalışma modu çalışıyor.


In [ ]:
print("TensorFlow version: {}".format(tf.__version__))
print("Keras version: {}".format(keras.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.4.1
Keras version: 2.3.1
Eager execution: True


### **Verileri Yükleme**

In [ ]:
#document = pd.read_csv('train_document.csv')
#print("train_document.csv dosyası içeri aktarıldı.")
#document.drop(['Unnamed: 0'], axis=1, inplace=True)

#news_texts = document['News']
#title_texts = document['Title']

In [ ]:
# document = pd.read_csv('Documents/documentRevV2.csv')
# document.drop(['Unnamed: 0'], axis=1, inplace=True)

# news_texts = document['News']
# title_texts = document['Title']

In [ ]:
documentPre = pd.read_csv('documentRevV2_3.csv')
documentPre.drop(['Unnamed: 0'], axis=1, inplace=True)
document = documentPre[:125000]
news_texts = document['News']
title_texts = document['Title']

series_news_texts = pd.Series(news_texts)
series_title_texts = pd.Series(title_texts)

## **Eğitim Öncesi Ön İşlemler**


---


Haber ve Başlıklar için Metin Ön İşleme Aşaması

Vektörizasyon, sekans uzunluklarının dolgulanması, vektörleri tensörlere dönüştürme ve dataset'in yığın halinde ayarlanması işlemleri...

In [ ]:
with open("TurkishStopWords.txt",mode="r") as tsw:
  stopWords = tsw.read()
  stopWords = stopWords.split("\n")

In [ ]:
def news_preprocess(row):
  # Sıra önemli
  # Tarihleri silmek için (05.11.1994) (5 Kasım 1994)
  row = re.sub("(([0-9]+(\.)[0-9]+(\.)[0-9]+)|([0-9]+.(ocak|şubat|mart|nisan|mayıs|haziran|temmuz|ağustos|eylül|ekim|kasım|aralık).[0-9]+)|([0-9]+.(ocak|şubat|mart|nisan|mayıs|haziran|temmuz|ağustos|eylül|ekim|kasım|aralık)))\S+", "", row, flags=re.IGNORECASE)
  row=re.sub(r"([0-9]+)", ' ', str(row))
  row = re.sub("([Hh][Aa][Bb][Ee][Rr][İi][Nn] [Dd][Ee][Vv][Aa][Mm][Iı])\W+"," ",str(row)) # 'Haberin Devamı' yazan haberleri temizler 
  row=re.sub("([Kk]aynak: [abcçdefghıijklmnoöprsştuüvyzABCÇDEFGHIİJKLMNOÖPRSŞTUÜVYZ])\w+", ' ', str(row)) 
  row=re.sub("([İi]şte [Dd]etaylar)\W+", ' ', str(row))
  row=re.sub(r"(([ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ]{2,})+ ([ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ]{2,})+ ([ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ]{2,}))\w+", ' ', str(row)) # Ara başlıkları silmek için.
  row=re.sub(r"(([ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ]{2,})+ ([ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ]{2,}))\w+", ' ', str(row)) # Ara başlıkları silmek için.
  row = re.sub("(İ)","i",str(row)) # Büyük İ sorunu için ASCII karşlığı yok
  row = re.sub("(I)","ı",str(row)) # Büyük I sorunu için ASCII karşlığı yok
  row=re.sub("(\\t)", ' ', str(row)) # Tab'ları tek boşluk yapmak için
  row=re.sub("(\\n)", ' ', str(row))
  row=re.sub("(\\r)", ' ', str(row))
  row = re.sub("(\s+)",' ',str(row)) # Birden fazla boşluğu tek boşluğa düşürmek için
  row=re.sub("(__+)", ' ', str(row))
  row=re.sub("(--+)", ' ', str(row))
  row=re.sub("(~~+)", ' ', str(row))
  row=re.sub("(\+\++)", ' ', str(row))
  row=re.sub("(\.\.+)", ' ', str(row))
  row=re.sub(r"[<>(){|}&©ø\[\]\'\",;:?~*!#$%‘’+-.=@`]", ' ', str(row)) # Karakter temizliği
  row=re.sub(r"[()]\S+", ' ', str(row)) # Parantez içini silmek için
  # Çok sayıları tek sayı biçimine düşürmek için
  row=re.sub(r"([0-9]+)", ' ', str(row))
  # row=re.sub("(###)\W+", '### ', str(row))
  row = re.sub("(\s+)",' ',str(row)) # Birden fazla boşluğu tek boşluğa düşürmek için
  row=re.sub(r"(\s+.\s+)\w+", ' ', str(row))
  row = re.sub("(\s+)",' ',str(row))
  row = row.strip() # Metinin sağ ve sol tarafındaki boşluğu silmek için
  row = row.lower()

  words_list = row.split()
  filtered_stop_words = [word for word in words_list if word not in stopWords]
  filtered_words = [word for word in filtered_stop_words if len(word)>2]

  row = " ".join(filtered_words)

  return row

In [ ]:
def title_preprocess(row):
  # Sıra önemli
  row = re.sub("(İ)","i",str(row)) # Büyük İ sorunu için ASCII karşlığı yok
  row = re.sub("(I)","ı",str(row)) # Büyük I sorunu için ASCII karşlığı yok
  row=re.sub("(\\t)", ' ', str(row)) # Tab'ları tek boşluk yapmak için
  row=re.sub("(\\n)", ' ', str(row))
  row=re.sub("(\\r)", ' ', str(row))
  row = re.sub("(\s+)",' ',str(row)) # Birden fazla boşluğu tek boşluğa düşürmek için
  row=re.sub("(__+)", ' ', str(row))
  row=re.sub("(--+)", ' ', str(row))
  row=re.sub("(~~+)", ' ', str(row))
  row=re.sub("(\+\++)", ' ', str(row))
  row=re.sub("(\.\.+)", ' ', str(row))
  row=re.sub(r"[<>(){|}&©ø\[\]\'\",;:?~*!#$%‘’+-.=@`]", ' ', str(row)) # Karakter temizliği
  row=re.sub(r"[()]\S+", ' ', str(row)) # Parantez içini silmek için
  # Çok sayıları tek sayı biçimine düşürmek için
  row=re.sub(r"([0-9]+)", '###', str(row))
  row=re.sub("(###)\W+", '### ', str(row))
  row = re.sub("(\s+)",' ',str(row)) # Birden fazla boşluğu tek boşluğa düşürmek için
  row=re.sub(r"(\s+.\s+)\w+", ' ', str(row))
  row = re.sub("(\s+)",' ',str(row))
  row = row.strip() # Metinin sağ ve sol tarafındaki boşluğu silmek için
  row = row.lower()
 
  return row

In [ ]:
# series_news_texts = pd.Series(news_texts)
# series_title_texts = pd.Series(title_texts)

In [ ]:
_ISTART_ = '<istart> '
_IEND_ = ' <iend>'

_TSTART_ = '<tstart> '
_TEND_ = ' <tend>'

tqdm.pandas(desc="Progress")

series_news_texts = series_news_texts.progress_apply(lambda x: _ISTART_ + news_preprocess(str(x)) + _IEND_).astype(str)
series_title_texts = series_title_texts.progress_apply(lambda x: _TSTART_ + title_preprocess(str(x)) + _TEND_).astype(str)

Progress: 100%|██████████| 120000/120000 [00:04<00:00, 27217.03it/s]


In [ ]:
# preparedData = {'News':series_news_texts, 'Title':series_title_texts}
# preparedSuDerDataset = pd.DataFrame(preparedData)
# preparedSuDerDataset.to_csv("preparedSuDerDataset2.csv")

**Dataset Train ve Validation Olarak Bölme**




---



In [ ]:
#Input ve Output'a göre vektörleri hazırlama
encoder_maxlen = 384
decoder_maxlen = 14

short_text = []
short_summary = []

for i in range(len(series_news_texts)):
    if(len(series_title_texts[i].split())<=decoder_maxlen and len(series_news_texts[i].split())<=encoder_maxlen):
        short_text.append(series_news_texts[i])
        short_summary.append(series_title_texts[i])

short_text_series = pd.Series(short_text)
short_summary_series = pd.Series(short_summary)

news_train, news_validation, title_train, title_validation = train_test_split(short_text_series, short_summary_series, test_size=0.1, shuffle=False)

In [ ]:
len(series_news_texts), len(short_text_series), len(series_title_texts), len(short_summary_series), 

(120000, 111200, 120000, 111200)

In [ ]:
# news_train, news_validation, title_train, title_validation = train_test_split(series_news_texts, series_title_texts, test_size=0.1)

**Metinleri tokenlere(jeton, vektör) Dönüştürme**

---



In [ ]:
# '<' ve '>' varsayılan belirteçlerden kaldırılamaz. Çünkü dekoder için hedef derlemimizi bu işaretler arasına aldık
# Gerekli olan filtreleme işlemini news_preprocess ve title_preprocess fonksionlarıyla yaptık
filters = '!"$%()*+,./:;=?@[\\]^_{|}~\t\n'

In [ ]:
#news_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters)
#title_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters)
news_tokenizer = keras.preprocessing.text.Tokenizer(filters=filters)
title_tokenizer = keras.preprocessing.text.Tokenizer(filters=filters)

In [ ]:
print("Haber metinlerinin vektör sözlüğü oluşturuluyor...")
news_tokenizer.fit_on_texts(news_train)
print("Haber metinlerinin vektör sözlüğü oluşturuldu.")

print("Haber başlıklarının vektör sözlüğü oluşturuluyor...")
# news_tokenizer.fit_on_texts(title_train)
title_tokenizer.fit_on_texts(title_train)
print("Haber başlıklarının vektör sözlüğü oluşturuldu.")
# --------------------------------------------------------------------
# print("Haber metinlerinin vektör sözlüğü oluşturuluyor...")
# news_tokenizer.fit_on_texts(series_news_texts)
# print("Haber metinlerinin vektör sözlüğü oluşturuldu.")

# print("Haber başlıklarının vektör sözlüğü oluşturuluyor...")
# news_tokenizer.fit_on_texts(series_title_texts)
# print("Haber başlıklarının vektör sözlüğü oluşturuldu.")

In [ ]:
print("Haber metinleri vektörleştiriliyor...")
inputs = news_tokenizer.texts_to_sequences(news_train)
print("Haber metinleri vektörleştirildi.")

print("Haber başlıkları vektörleştiriliyor...")
# targets = news_tokenizer.texts_to_sequences(title_train)
targets = title_tokenizer.texts_to_sequences(title_train)
print("Haber başlıkları vektörleştirildi.")

In [ ]:
# news_tokenizer.texts_to_sequences(["<istart>, <iend>"]), title_tokenizer.texts_to_sequences(["<tstart>, <tend>"])

In [ ]:
encoder_vocab_size = len(news_tokenizer.word_index) + 1 
decoder_vocab_size = len(title_tokenizer.word_index) + 1

print('Encoder sözlük sayısı: {}\nDecoder sözlük sayısı: {}'.format(encoder_vocab_size, decoder_vocab_size))

Encoder sözlük sayısı: 410820
Decoder sözlük sayısı: 56115


**Haber ve başlık metinlerinin ayrı ayrı uzunluklarını tanımlama**

In [ ]:
news_train_lengths = pd.Series([len(x.split()) for x in news_train])
title_train_lengths = pd.Series([len(x.split()) for x in title_train])

news_validation_lengths = pd.Series([len(x.split()) for x in news_validation])
title_validation_lengths = pd.Series([len(x.split()) for x in title_validation])

In [ ]:
# print("Train Haber metni istatistikleri")
# news_train_lengths.describe()

In [ ]:
# print("Validation Haber metni istatistikleri")
# news_validation_lengths.describe()

In [ ]:
# print("Train Başlık metni istatistikleri")
# title_train_lengths.describe()

In [ ]:
# print("Validation Başlık metni istatistikleri")
# title_validation_lengths.describe()

In [ ]:
ser = news_train_lengths
tit = title_train_lengths

In [ ]:
# fig = plt.figure(dpi=1200)
# ser.plot.kde(bw_method=0.3,grid=True).set_xlim(0,800)

In [ ]:
# ser.plot.kde(bw_method=0.3,grid=True).set_xlim(0,256)

In [ ]:
# fig = plt.figure(dpi=1200)
# tit.plot.kde(bw_method=0.4,grid=True).set_xlim(0,16)

In [ ]:
# plt.plot(news_train_lengths,)
# plt.title("Haber Gövdesi Dağılım Grafiği")
# plt.xlabel("Haber Sayısı")
# plt.ylabel("Sözcük Sayısı")
# plt.show()

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs,
    maxlen=encoder_maxlen,
    padding='post',
    truncating='post')

targets = tf.keras.preprocessing.sequence.pad_sequences(
    targets,
    maxlen=decoder_maxlen,
    padding='post',
    truncating='post')

In [ ]:
# tf.cast almış olduğu x vektör dizisini (bu kod bloğunda inputs ve targets) tensör olarak döndürür. Tensörler numpy() dizileri gibi düşünülebilir
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [ ]:
BUFFER_SIZE = 5000
BATCH_SIZE = 128

**Eğitim için burada bi adımlama tekniği kullanacağımız için bölümleme içeriklerini de değiştireceğiz.**

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 384]), TensorShape([64, 14]))

# **Transformatör Mimarisi**
---
Pozisyonal kodlama, öz dikkat mekanizması, çok başlı dikkat mekanizması vs...

**RNN'den farklı olarak kelimeler arasında konum kavramı eklemek için konumsal kodlama, bu yönsüzdür.**

In [ ]:
def get_angles(position, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
  return position * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(
      np.arange(position)[:, np.newaxis],
      np.arange(d_model)[np.newaxis, :],
      d_model)

  # Dizideki bütün çift(2i) indeksli satırlara sinüs fonksiyonu uygulanır
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # Dizideki bütün tek(2i+1) indeksli satırlara kosinüs fonksiyonu uygulanır
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

**Maskeleme**


*   "pad" sekanslarını maskelemek için dolgu maskesi. (Sekansımızın hangi kısmı sonradan dolgu olarak eklenmiş)

*  Mevcut sözcüklerin öz dikkat mekanizmasını kullanarak gelecek sözcük üretimine katkı sağlaması için ileri görüş maskelemesi sağlar.     

In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return seq[:,tf.newaxis, tf.newaxis,:]

In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask

In [ ]:
def create_masks(inp, tar):
  enc_padding_mask = create_padding_mask(inp)
  dec_padding_mask = create_padding_mask(inp)

  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

### **Modeli Oluşturma**



**Scaled Dot Product Attention (Ölçekli Nokta Çarpımı)**

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  matmul_qk = tf.matmul(q, k, transpose_b=True)

  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

  output = tf.matmul(attention_weights, v)
  return output, attention_weights

**Multi-Head Attention (Çok Başlı Dikkat Mekanizması)**

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0
    
    
    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  #x bir tensördür.
  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)
    k = self.wk(k)
    v = self.wv(v)

    q = self.split_heads(q, batch_size)
    k = self.split_heads(k, batch_size)
    v = self.split_heads(v, batch_size)
    
    scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
    output = self.dense(concat_attention)

    return output, attention_weights

**Feed Forward Network (İleri Besleme Ağı)**

(Transformatör mimarisindeki katmanlarda yaptığımız işlemleri bir sonraki katmana iletmek için)

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([tf.keras.layers.Dense(dff, activation='relu'), tf.keras.layers.Dense(d_model)])

**Transformatör Mimarisi Encoder(Kodlayıcı) Temel Birimi**

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):
    attn_output, _ = self.mha(x, x, x, mask)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x+attn_output)

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1+ffn_output)

    return out2

**Transformatör Mimarisi Decoder(Kod Çözücü) Temel Birimi**

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)

    ffn_output = self.ffn(out2)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)

    return out3, attn_weights_block1, attn_weights_block2

**Çoklu Kodlayıcı Katmanlarından Oluşan Kodlayıcı (N x Encoder)**

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()
    
    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = Embedding(input_vocab_size, d_model) #trainable = True
    
    self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):
    seq_len = tf.shape(x)[1]

    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
  
    return x

**Çoklu Kod Çözücü Katmanlarından Oluşan Kod Çözücü (N x Decoder)**

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate = 0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = Embedding(target_vocab_size, d_model) # trainable=True
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model,tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

    return x, attention_weights

**Transformatör Sınıfı**

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
    self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
    enc_output = self.encoder(inp, training, enc_padding_mask)
    dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)

    return final_output, attention_weights

# **Eğitim**

## **Kontrol Paneli**

---
(Hiperparametreler, optmizer, loss fonkiyonu ve transformatör ayarları)


In [ ]:
# Hiperparametreler
num_layers = 6
d_model = 512
dff = 2048
num_heads = 8
EPOCHS = 25

**Özel Öğrenme Oranıyla Adam Optimizer**

In [ ]:
# Serileştirilebilir Öğrenme Oranı Düşüş Programı.
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

**Kayıp Fonksiyonu Tanımlama ve Diğer Ölçümler**

In [ ]:
learning_rate = CustomSchedule(d_model)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
def accuracy_function(real, pred):
  pred = tf.cast(tf.argmax(pred[:][:], axis=-1), tf.int32)
  accuracies = tf.equal(real, pred)

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)

  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

**Transfomatör**

In [ ]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads,
    dff,
    encoder_vocab_size,
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size)

## **Eğitim Adımı Fonksiyonu**

### **Kayıt ve Tensorboard Config**

In [ ]:
checkpoint_path = "checkpoints/rev2/transformers/"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Son kayıt noktası düzenlendi!!')


# checkpoint_path_epoch20 = "checkpoints/test8/ckpt-10"
# ckpt.restore(checkpoint_path_epoch20)

In [ ]:
ckpt_manager.latest_checkpoint

In [ ]:
dataset

<BatchDataset shapes: ((None, 384), (None, 14)), types: (tf.int32, tf.int32)>

In [ ]:
logdir = "checkpoints/rev2/transformers/"
writer = tf.summary.create_file_writer(logdir)

### **Ana Eğitim Kod Bloğu**

In [ ]:
 @tf.function
 def train_step(inp, tar):
   tar_inp = tar[:, :-1]
   tar_real = tar[:, 1:]

   enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

   with tf.GradientTape() as tape:

     predictions, _ = transformer(inp,tar_inp,True,enc_padding_mask,combined_mask,dec_padding_mask)
     loss = loss_function(tar_real, predictions)

   gradients = tape.gradient(loss, transformer.trainable_variables)    
   optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

   train_loss(loss)
   train_accuracy(accuracy_function(tar_real, predictions))
   # train_accuracy(tar_real, predictions)

In [ ]:
 train_total_time = []

 with writer.as_default():
   for epoch in range(EPOCHS):

     if epoch == 0:
       print("Eğitim aşaması başladı...")
    
     start = time.time()

     train_loss.reset_states()
     train_accuracy.reset_states()
  
     for (batch, (inp, tar)) in enumerate(dataset):
       train_step(inp, tar)

       if batch % 100 == 0: # if batch % 50 == 0:
         print("Epoch: {} Batch: {} Doğruluk: {:.4f} Kayıp: {:.4f}".format(epoch+1, batch, train_accuracy.result(), train_loss.result()))

     tf.summary.scalar('Loss', data=float(train_loss.result()), step=epoch)
     tf.summary.scalar('Accuracy', data=float(train_accuracy.result()), step=epoch)
     writer.flush()
    
    #  if (epoch + 1) % 5 == 0:
    #      ckpt_save_path = ckpt_manager.save()
    #      print("{}. epoch'ta eğitim {} olarak kayıt edildi.".format(epoch+1, ckpt_save_path))

     ckpt_save_path = ckpt_manager.save()
     print("{}. epoch'ta eğitim {} olarak kayıt edildi.".format(epoch+1, ckpt_save_path))
  
     print("Epoch: {} Doğruluk: {:.4f} Kayıp: {:.4f}".format(epoch+1, train_accuracy.result(), train_loss.result()))    
  
     epoch_duration = time.time() - start
     train_total_time.append(epoch_duration)
     print("Şuan ki epoch için geçen toplam süre: {:.4f} dk".format(epoch_duration/60))
     print("Toplam geçen süre: {:.4f} dk".format(sum(train_total_time)/60))

   if epoch == EPOCHS-1:
     ckpt_save_path = ckpt_manager.save()
     print("Eğitim sonu model {} olarak kayıt edildi.".format(ckpt_save_path))
     print("Eğitim aşaması bitti...")

In [ ]:
# ckpt.save("model_train")
# os.listdir()
# file.download("model_train-15.data-00000-of-00001")

# **Sonuç**
Kod çözücüde bir zamanda bir kelimeyi tahmin etmek ve onu çıktıya eklemek;

daha sonra kod çözücüye dizinin(sekans) tamamını girdi olarak almak ve \<stop> veya maxlen ile karşılaşana kadar aynı işleme devam etmek.
  

In [ ]:
# Üretimsel Dekoder (Transformatör mimarisinin üretim tarafında kullandığı dekoder bloğu)
def evaluate(input_document):
  # inp_sentence = _ISTART_ + news_preprocess(input_document) + _IEND_
  inp_sentence = input_document

  input_document = news_tokenizer.texts_to_sequences([input_document])
  input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')
  encoder_input = tf.expand_dims(input_document[0], 0)

  decoder_input = [title_tokenizer.word_index["<tstart>"]]
  output = tf.expand_dims(decoder_input, 0)

  for i in range(decoder_maxlen):
    enc_padding_mask, combined_mask, decoder_padding_mask = create_masks(encoder_input, output)
        
    predictions, attention_weights = transformer(
        encoder_input,
        output,
        False,
        enc_padding_mask,
        combined_mask,
        decoder_padding_mask)
        
    predictions = predictions[ : ,-1:, :]
    
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        
    if predicted_id == title_tokenizer.word_index["<tend>"]:
      return tf.squeeze(output, axis=0), attention_weights
    
    output = tf.concat([output, predicted_id], axis=-1)
        
  return tf.squeeze(output, axis=0), attention_weights

In [ ]:
# Başlık üretmek için yazılan foksiyon input_document parametresi haber gövdesini temsil eder, return fonksiyonu ile haber gövdesi için üretilen başlık döndürülür.
def generate_title(input_document):
  generated_title = evaluate(input_document=input_document)[0].numpy()
  generated_title = np.expand_dims(generated_title[1:],0) 

  return title_tokenizer.sequences_to_texts(generated_title)[0]

In [ ]:
generated_titles = []

for i, news in enumerate(news_validation):
  generated_title = generate_title(news)
  generated_titles.append(generated_title)

generated_titles_series = pd.Series(generated_titles)

In [ ]:
dict_validation_generated = {"News":news_validation, "OriginalTitle":title_validation, "Predictions":generated_titles}
df_validation_generated = pd.DataFrame(dict_validation_generated)
df_validation_generated.to_csv("GeneratedTitlesTransformers.csv")

## **Computing Score**

### **BLEU**

In [ ]:
!pip install --upgrade bleu
from bleu import list_bleu

In [ ]:
ref = ["""thy nin yolcu sayısı ### milyona ulaştı"""]
hyp = ["""THY yolcu sayısını 9.2 milyona çıkardı"""]

In [ ]:
list_bleu([ref], hyp)



---



In [ ]:
import nltk

In [ ]:
hypothesis = "### terörist etkisiz hale getirildi".split()
reference = title_preprocess("Bitlis'te 6 teröristin öldürüldüğü operasyonda çok sayıda silah mühimmat ele geçirildi").split()
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,auto_reweigh=True)
print(BLEUscore*100)



---



In [ ]:
# two references for one document
from nltk.translate.bleu_score import corpus_bleu
references = [["".split()]]
candidates = ["".split(), "".split()]
score = corpus_bleu(references, candidates)
print(score)

### **ROUGE**

In [ ]:
!pip install 0 0 

In [ ]:
from rouge_metric import PyRouge

By default, sentences are separated by '\n' and tokens are separated by white space in a document. This tokenization process can be further customized. For example,

In [ ]:
# # Template
# from rouge_metric import PyRouge

# # Pre-process and tokenize the summaries as you like
# hypotheses = [
#     ['how are you'.split(), 'i am fine'.split()],                       # document 1: hypothesis
#     ['it is fine today'.split(), 'we won the football game'.split()],   # document 2: hypothesis
# ]
# references = [[
#     ['how do you do'.split(), 'fine thanks'.split()],   # document 1: reference 1
#     ['how old are you'.split(), 'i am three'.split()],  # document 1: reference 2
# ], [
#     ['it is sunny today'.split(), 'let us go for a walk'.split()],  # document 2: reference 1
#     ['it is a terrible day'.split(), 'we lost the game'.split()],   # document 2: reference 2
# ]]

# # Evaluate on tokenized documents
# rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
#                 rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
# scores = rouge.evaluate_tokenized(hypotheses, references)
# print(scores)

In [ ]:
# hypotheses = ["kavga ### can alan maganda vurdu".split()]
# references = ["Kavga sebebi sudan faturası ise iki can".split()]

In [ ]:
# print(hypotheses)
# print(references)

In [ ]:
# references = [title.split() for title in title_texts]

In [ ]:
# hypotheses20 = [generate_title(news_preprocess(news)).split() for news in news_texts]

In [ ]:
# hypotheses20 = news_texts.progress_apply(lambda x: generate_title(news_preprocess(str(x))) if i < range(25000))

In [ ]:
7poıuytğü
hypotheses25 = []
references = []

# tqdm.pandas(desc="Progress")
# for i in tqdm(range(25000,52489)):
#   hypotheses20.append(generate_title(news_preprocess(news_texts[i])).split())
#   references.append(title_texts[i].split())

In [ ]:
dict_rouge = {"Tahmin - 25 Epoch":hypotheses25}

In [ ]:
df = pd.DataFrame(dict_rouge)

In [ ]:
df.to_csv("s25k25e.csv")

In [ ]:
for i in tqdm(range(25000,52489)):
  hypotheses25.append(generate_title(news_preprocess(news_texts[i])))

100%|██████████| 27489/27489 [9:17:10<00:00,  1.22s/it]
